In [1]:
import sqlite3
import os
import dotenv
from tqdm import tqdm

dotenv.load_dotenv()

conn = sqlite3.connect(os.getenv("DB_FILE"))
cur = conn.cursor()

In [2]:
licee = cur.execute("SELECT id_liceu, nume_afisat, rank, address, website, facebook, instagram, wikipedia FROM licee").fetchall()

for liceu in tqdm(licee):
  cur.execute("INSERT INTO institutii(id, nume, rank, adresa, website, facebook, instagram, wikipedia) VALUES(?, ?, ?, ?, ?, ?, ?, ?)", liceu)
conn.commit()

100%|██████████| 1664/1664 [00:00<00:00, 59336.37it/s]


In [2]:
scoli = cur.execute("SELECT id_scoala, nume_afisat, cod_siiir FROM scoli").fetchall()

# conn.close()
conn = sqlite3.connect(os.getenv("DB_FILE"))
cur = conn.cursor()

for scoala in tqdm(scoli):
  exists = cur.execute("SELECT * FROM institutii WHERE id = ?", (scoala[0],)).fetchone()

  try:
    if not exists:
      cur.execute("INSERT INTO institutii(id, nume) VALUES(?, ?)", (scoala[0], scoala[1]))
  except Exception as e:
    print(scoala)
    conn.close()
    raise e
conn.commit()

100%|██████████| 6206/6206 [00:00<00:00, 52461.77it/s]


In [11]:
import unidecode
import sys

sys.path.append("..")

from judete import get_county_code
from unification import cannonicalize_name


siiir = cur.execute("SELECT cod, denumire, judet, tip_unitate FROM siiir").fetchall()

siiir_from_id = {}
for s in siiir:
  cod, denumire, judet, tip_unitate = s

  if tip_unitate != "Unitate de învățământ":
    continue

  cod_judet = get_county_code(judet)
  id = cannonicalize_name(denumire, cod_judet, True)
  # if id in siiir_from_id:
  #   print(f"Duplicate {id} {denumire} {cod_judet} {siiir_from_id[id]} -> {cod}")

  siiir_from_id[id] = cod

print(len(siiir_from_id))

institutii = cur.execute("SELECT id FROM institutii").fetchall()

for (id, ) in tqdm(institutii):
  if id in siiir_from_id:
    cur.execute("UPDATE institutii SET cod_siiir = ? WHERE id = ?", (siiir_from_id[id], id))

conn.commit()


17688


100%|██████████| 6999/6999 [00:00<00:00, 205598.24it/s]


In [17]:

from judete import get_county_code, judete_dupa_cod

institutii = cur.execute("SELECT id, adresa, website, cod_siiir FROM institutii").fetchall()

for (id, adresa, website, cod_siiir) in tqdm(institutii):
  if cod_siiir is not None:
    siiir = cur.execute("SELECT strada, nr_strada, localitate, cod_postal, website, telefon, cod_fiscal FROM siiir WHERE cod = ?", (cod_siiir,)).fetchone()

    if not adresa:
      adresa_siiir = siiir[0].strip(' ,.')
      if siiir[1].strip() not in ["", "-", ".", "x"]:
        adresa_siiir += " " + siiir[1].strip(' ,.')
      if siiir[2] is not None:
        adresa_siiir += ", " + siiir[2].title()
      if siiir[3] is not None:
        adresa_siiir += ", " + siiir[3]

      cur.execute("UPDATE institutii SET adresa = ? WHERE id = ?", (adresa_siiir, id))
    
    if not website and siiir[4]:
      cur.execute("UPDATE institutii SET website = ? WHERE id = ?", (siiir[4], id))
    
    if siiir[5]:
      cur.execute("UPDATE institutii SET telefon = ? WHERE id = ?", (siiir[5], id))
    
    if siiir[6]:
      cur.execute("UPDATE institutii SET cod_fiscal = ? WHERE id = ?", (siiir[6], id))

conn.commit()

  0%|          | 0/6999 [00:00<?, ?it/s]

100%|██████████| 6999/6999 [00:00<00:00, 7227.18it/s]


In [20]:
conn.close()
conn = sqlite3.connect(os.getenv("DB_FILE"))
cur = conn.cursor()

scoli = cur.execute(
    "SELECT id_scoala FROM scoli"
).fetchall()

for id_scoala in (scoli):
    cur.execute(
        "UPDATE institutii SET gimnaziu = 'da' WHERE id = ?",
        (id_scoala),
    )

print(f"Updated {len(scoli)} scoli.")

liceu = cur.execute(
    "SELECT id_liceu FROM licee"
).fetchall()

for id_liceu in (liceu):
    cur.execute(
        "UPDATE institutii SET liceu = 'da' WHERE id = ?",
        (id_liceu),
    )

print(f"Updated {len(liceu)} licee.")

conn.commit()

Updated 6206 scoli.
Updated 1664 licee.
